# Chapter 3

Sampling from the posterior - we have computers, why do we need a closed form for the posterior distribution when we can have a massive numpy array instead?

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats
import seaborn as sns

import pybayes

sns.set(style='whitegrid')

## Sampling from the grid-approximate posterior

In [ ]:
# using the example from chapter 2 of 9 attempts, 6 successes.
posterior = pybayes.utils.grid_approximate_binomial(n=9,
                                                    k=6,
                                                    grid_size=1000,
                                                    prior=None,
                                                    plot=False)

In [ ]:
pybayes.utils.plot_nicely(x_vals=posterior[:,0], y_vals=posterior[:,1])

In [ ]:
# sample this distribution.
samples = np.random.choice(posterior[:,0],
                           size=int(1e4),
                           p=posterior[:,1],
                           replace=True)

In [ ]:
# show the sampling - plot the sequence, then the density
fig, axes = plt.subplots(nrows=2, figsize=(5,10))
sns.scatterplot(x=np.arange(len(samples)), y=samples, alpha=0.2, ax=axes[0])
axes[0].set_ylim(0,1)
axes[0].set_xlabel('Sequence number')
axes[0].set_ylabel('Sampled p')

sns.histplot(x=samples, ax=axes[1], element='poly', fill=False)
axes[1].set_xlim(0,1)
axes[1].set_xlabel('Sampled p')
axes[1].set_ylabel('Frequency density')
plt.show()


Once we have samples from the posterior, we can do things we actually care about, such as point estimates, and compatability intervals (McElreath dislikes the phrase 'confidence interval'). 

### Intervals

In [ ]:
# E.g. 1: what  is the probability that p < 0.5 given our data
# and binomial model?

# from our grid approximation
grid_approx_p_of_half = posterior[posterior[:,0] < 0.5, :][:, 1].sum()
print(f'grid result: {grid_approx_p_of_half: .3f}')

# from the samples (easier in general)
sampled_p_of_half = sum(samples < 0.5) / len(samples)
print(f'sampled result: {sampled_p_of_half: .3f}')


In [ ]:
# E.g 2: what is the 10%-90% interval for our posterior p?  We can get
# this trivially from quantiles

print('10% - 90%')
print(f'{np.quantile(samples, 0.1):.2f} - {np.quantile(samples,0.9):.2f}')

NB the percentile intervals are nice summaries of the distribution, unless the distro is highly skewed, but they're not ideal for inference.

In [ ]:
# TODO code for the PI and HDPI

### Point estimates

Note - you don't really need one. The Bayesian parameter estimate is the distribution you just computed, anything else is a discarding of useful information. But if you want one, one option is the maximum a posteriori (MAP) estimate, the mode of the distribution.

For the case where we have three successes in three trials, the distribution of p looks like:

In [ ]:
posterior_three_successes = pybayes.utils.grid_approximate_binomial(n=3,
                                                    k=3,
                                                    grid_size=1000,
                                                    prior=None,
                                                    plot=True)
# sample this distribution.
samples_three_successes = np.random.choice(posterior_three_successes[:,0],
                           size=int(1e4),
                           p=posterior_three_successes[:,1],
                           replace=True)

In [ ]:
# from the grid approximation:
mode = x_for_max_y = posterior_three_successes[np.argmax(posterior_three_successes[:, 1]), 0]
print(f'MAP from grid: {mode:.2f}')
# from the samples